In [4]:
!nvidia-smi

Sat Feb  7 10:07:50 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
from google.colab import drive
import os

# 1. Mount Drive (This will ask for a permission code)
drive.mount('/content/drive')

# 2. Define the path to your files in the cloud
# (Assumes you uploaded them to the main "My Drive" folder)
train_path = '/content/drive/MyDrive/train.bin'
val_path = '/content/drive/MyDrive/val.bin'

# 3. Verify the bridge is working
if os.path.exists(train_path):
    print(f"✅ SUCCESS: Found {train_path}")
else:
    print(f"❌ FAIL: Could not find files. Make sure they are in the root of Google Drive.")

Mounted at /content/drive
✅ SUCCESS: Found /content/drive/MyDrive/train.bin


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import time
import os

# --- 1. CONFIGURATION ---
# ⚠️ CHANGE THIS if your files are in a subfolder!
# If files are in "My Drive/smoltok", change this to: '/content/drive/MyDrive/smoltok'
data_dir = '/content/drive/MyDrive' 

# Hyperparameters (Tuned for T4 GPU)
batch_size = 64        
block_size = 256       
max_iters = 5000       
eval_interval = 500    
learning_rate = 3e-4   
eval_iters = 200
n_embd = 384           
n_head = 6             
n_layer = 6            
dropout = 0.2          

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- 2. LOAD DATA FROM DRIVE ---
print(f"⚙️ Device: {device}")
if device == 'cuda':
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    torch.backends.cuda.matmul.allow_tf32 = True 
    torch.backends.cudnn.allow_tf32 = True

# Construct full paths
train_path = os.path.join(data_dir, 'train.bin')
val_path = os.path.join(data_dir, 'val.bin')

# Verify files exist
if not os.path.exists(train_path):
    print(f"❌ Error: Could not find 'train.bin' at: {train_path}")
    print("   -> Did you upload it to the MAIN folder of Google Drive?")
    print("   -> If it's in a subfolder, update 'data_dir' at the top of this script.")
    exit()

print(f"✅ Found data at: {train_path}")

# Load Data
train_data = np.fromfile(train_path, dtype=np.uint16)
val_data = np.fromfile(val_path, dtype=np.uint16)

vocab_size = max(np.max(train_data), np.max(val_data)) + 1
print(f"📚 Vocab Size: {vocab_size}")

# --- 3. DATA LOADER ---
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# --- 4. THE GPT MODEL ---
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   
        q = self.query(x) 
        wei = q @ k.transpose(-2, -1) * C**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x) 
        out = wei @ v 
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) 
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) 
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x) 
        logits = self.lm_head(x) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

# --- 5. EXECUTION ---
if __name__ == '__main__':
    model = GPT()
    model = model.to(device)
    
    print("🚀 Compiling model for T4 speed...")
    model = torch.compile(model)

    print(f"🧠 Parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scaler = torch.amp.GradScaler('cuda')

    print(f"🏎️  Training started (Batch Size: {batch_size})...")
    start_time = time.time()

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        xb, yb = get_batch('train')
        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            logits, loss = model(xb, yb)

        optimizer.zero_grad(set_to_none=True)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    end_time = time.time()
    print(f"🏁 Finished in {(end_time - start_time)/60:.2f} minutes.")

    # Save to Drive so you don't lose it!
    save_path = os.path.join(data_dir, 'model_day10.pt')
    torch.save(model.state_dict(), save_path)
    print(f"💾 Model saved to Google Drive: {save_path}")

⚙️ Device: cuda
🎮 GPU: Tesla T4
❌ Error: 'train.bin' not found. Please upload it to the Colab files sidebar!


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


FileNotFoundError: [Errno 2] No such file or directory: 'train.bin'

: 